# 장해


In [ ]:
#-*-coding:utf-8-*-

# 연습문제 8-3: 유튜브 댓글 수집하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("연습문제 : 유투브 영상의 댓글 수집하기")
print("=" *80)


query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")



# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
#path = "C:\\Temp\\chromedriver_240\\chromedriver.exe"
driver = webdriver.Chrome("C:\\Temp\\chromedriver_240\\chromedriver.exe",options=chrome_options)

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

#검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if reple_cnt < 20 :
        page_cnt = 1 
else :
        page_cnt = math.ceil(cnt/20)
        
if cnt > 20 :
           
        i = 1
        while ( i <=  page_cnt) :
            print("화면을 %s 회 아래로 스크롤 합니다" %i)
            scroll_down(driver)
            time.sleep(1)
            i += 1
      

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
          
          item.append(i['href'])
          count += 1

          if count == cnt :
              break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
          url_cnt += 1
          url = 'https://www.youtube.com/' +item[x]
          full_url.append(url)
          if url_cnt == cnt :
            break


play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]

wb=openpyxl.Workbook( )
wb.save(fx_name)


for y in range(0,len(full_url)) :
      driver.get(full_url[y])
      time.sleep(10)
      print("\n")
      print("\n")
      print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

      i = 1
      while ( i <=  page_cnt) :
      #print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(10)
        i += 1
      
      html = driver.page_source
      soup2 = BeautifulSoup(html, 'html.parser')
        
      t_count_0 = soup2.select('#count')
      t_count_1 = t_count_0[1].get_text()
      t_count_2 = t_count_1.replace(",","")
      t_count_3 = re.search("\d+",t_count_2)
      t_count_4 = int(t_count_3.group())

      t_title_1 = soup2.select('#info-contents')
      t_title_2 = t_title_1[0].find('h1').get_text()
      t_title_3 = t_title_2.translate(bmp_map).replace("\n","")
        
        
      try:
        t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()  #.get_text( ).replace("\n","")
      
      except AttributeError:
        continue
      
      else:
        t_view_2 = t_view_1.replace("," ,"")
        t_view_3 = re.search("\d+",t_view_2)
        print(t_view_1)
        t_view_4 = int(t_view_3.group())
        

#       t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()#.get_text( ).replace("\n","")
#       t_view_2 = t_view_1.replace("," ,"")
#       t_view_3 = re.search("\d+",t_view_2)
#       print(t_view_1)
#       t_view_4 = int(t_view_3.group())

      print("=" *80)
      print("%s 번째 동영상의 조회수는 %s 회 이고 수집할 댓글은 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
      print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
      print("=" *80)
      print("\n")
      print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
      print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
      print("\n")

 #Step 5. 화면을 스크롤해서 아래로 이동합니다

      #page_cnt = math.ceil(t_view_4 / 19)

      i = 1
      while ( i <=  page_cnt+1) :
        print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
        scroll_down(driver)
        time.sleep(0.5)
        i += 1

      #Step 6. 내용을 수집합니다

      time.sleep(2)

      html = driver.page_source
      soup3 = BeautifulSoup(html, 'html.parser')
        
      count = 0
      d2 = 0

      reple_result = soup3.select('#comments > #sections > #contents')

      for a in range(0,reple_cnt+1) :
              count += 1
              
              f = open(ff_name, 'a',encoding='UTF-8')
              f.write("\n")
              f.write("------------------------------------------------------------------"+"\n")

             # 댓글 작성자
              try : 
                writer = reple_result[0].select("#header-author > #author-text")[a].get_text().replace("\n","").strip()
              except IndexError :
                break
              else :
                print("\n")
                print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
                print("-" *70)

                # 유튜브 URL 주소
                print("1.URL 주소:",full_url[y])
                f.write("1.유투브 URL주소: " + full_url[y] + "\n")
                url2.append(full_url[y])
                
                print("2.댓글 작성자명:", writer)
                f.write("2.댓글 작성자명: " + writer + "\n")
                writer2.append(writer)
              
                # 댓글 작성일자
                wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")
            
                print("3.댓글 작성일자:",wdate)
                f.write("3.댓글 작성일자: " + wdate + "\n")
                wdate2.append(wdate)
              
                # 댓글 내용
                reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")
            
                reple2=reple1.translate(bmp_map).replace("\n","")
                print("4.댓글 내용:",reple2)
                f.write("4.댓글 내용: " + reple2 + "\n")
                reple3.append(reple2)
                
              
              f.close( )
              
              if count == reple_cnt :
                  break
              
      time.sleep(2)       
      play_cnt += 1                

 #step 9. 엑셀 형태로 저장하기
    
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)

driver.close( )

 

연습문제 : 유투브 영상의 댓글 수집하기
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 장해
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:200
3.각 동영상에서 추출할 댓글은 몇건입니까?: 159
4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\temp\):C:\\temp\\장해\\


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^
화면을 1 회 아래로 스크롤 합니다
화면을 2 회 아래로 스크롤 합니다
화면을 3 회 아래로 스크롤 합니다
화면을 4 회 아래로 스크롤 합니다
화면을 5 회 아래로 스크롤 합니다
화면을 6 회 아래로 스크롤 합니다
화면을 7 회 아래로 스크롤 합니다
화면을 8 회 아래로 스크롤 합니다
화면을 9 회 아래로 스크롤 합니다
화면을 10 회 아래로 스크롤 합니다


검색된 영상 중 200 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
댓글 159개
1 번째 동영상의 조회수는 39443 회 이고 수집할 댓글은 총 159개 입니다
1 번째 동영상의 제목은 사고후 후유장해보험금담보 100% 활용하기 (주위에 수술하신분 필수로 공유하세요)(95회) 입니다


1 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


1 번째 영상의 1 번째  댓글 
-----------------------------------

3.댓글 작성일자: 6개월 전
4.댓글 내용: 굿


1 번째 영상의 28 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 최수현
3.댓글 작성일자: 6개월 전
4.댓글 내용: 얼마전 아내가.계단어서 넘어저발목뼈가 골절되어 수술후 입원중인데후유장애 진단금신청가능한가요


1 번째 영상의 29 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 심옥희
3.댓글 작성일자: 2개월 전
4.댓글 내용: 제가 일하다가 가운데 손가락 절단됩습니다 약 한마디가량 그럼 몇프로죠?


1 번째 영상의 30 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: Jeongsook Hwang
3.댓글 작성일자: 2개월 전
4.댓글 내용: 안녕하세요작년 9월 사다리에서 떨어져 안면 안와골절 수술후 수술한 왼쪽 눈밑부터 웃입술 윗부분 코반쪽이 지금까지 마비인데 후유장해진단금 청구할수 있는지요수술은 대학병원 안과에서 했습니다


1 번째 영상의 31 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 김정임
3.댓글 작성일자: 4개월 전
4.댓글 내용: 쓸개제거수술 했는



1 번째 영상의 56 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 워니원TV
3.댓글 작성일자: 2개월 전
4.댓글 내용: 17년 11월 만성 아킬레스건염이 운동 중 부분 파열되어 봉합 수술 하였는데 올해 5.7 계단에서 심하게 접질러 복숭아뼈 쪽 인대와 뼈가 골절 되어 핀하나를 박는 수술을 하였는데 혹시 후유장해로 판정받아 보험금 수령가능할가요??


1 번째 영상의 57 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 88794 wsdsf
3.댓글 작성일자: 2개월 전
4.댓글 내용: 제가 가입한지 한달전에 정신과 두번 방문에 약을 29일치를 탔습니다그리고 의료실비를 고지안하고 가입후다시 세달후 정신과에 가서 약을탔습니다이경우에는 최근 정신과간후 5년동안 정신과를 안가도 이 실비 살릴수있을까요?


1 번째 영상의 58 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 김현섭
3.댓글 작성일자: 2개월 전
4.댓글 내용: 담낭절제술은 질병후유장해 받을수 있나요?


1 번째 영상의 59 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글

3.댓글 작성일자: 5개월 전
4.댓글 내용: 교통사고로 인해 허리와머리가 아파서 병원에 갔습니다 머리는 뇌출혈 이라고하여주사와 약으로 회복이 된듯한 데 허리가 아파서 주사와 침으로 약으로 치료를 해도 너무 힘들어요 그래서 병원에 갔더니 입원을 했지만 그래도 계속 허리통증이 심하게 아프고 해도 일 때문에 일주일 만에 퇴원을 하였습니다 그리고  다음날 다시 병원에 가서 제 보험 으로MRI 찍어 봤습니다 그런데 바로 수술을 해야 한다고 합니다  그래서 사진 가지고 다른 병원에 갔습니다 그 병원 도 수술 외에는 방법이 없다고 합니다 퇴행성디스크 협착증이 너무 심하다네요   자동차보험 으로 접수하면  치료를 소홀히 하는게 아닌가 해서 지금은 제 보험 으로 2주째 한의원 가서 치료를 받고 있습니다 12월에 서울 대학 병원 에 예약을 했는데 자동차보험 에서 수술 비용 과 보상을 받을 수 있는 지 궁금하네요 연락드리겠습니다 010-2886-7732입니다


1 번째 영상의 84 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 홍지성
3.댓글 작성일자: 5개월 전
4.댓글 내용: 회사에서 일하다가 중지손가락인대손상으로 6개월 산재받앗구 다시일한지 4개월째인데요...주먹이 확쥐어지지않는다구 의사분이말하셧는데요산재받앗는데 후유장해되는지요???


1 번째 영상의 85 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=PMl5M-ED6j4
2.댓글 작성자명: 정아영
3.댓글 작성일자: 4주 전
4.댓글 내용: 안녕하세요?유익한 정보 감사드립니다.후유장애에 대해 말씀해 주셨는데요,15년전쯤 호신술 수업도중 상

3.댓글 작성일자: 4개월 전(수정됨)
4.댓글 내용: 신랑 10년전 쯤  교통사고로 추간판탈출증 수술받고교통사고 손해배상  5년 한시장애로 판단되서 합의금 받고,개인보험사에는  장애5급 진단서로 보험금 청구,지자체 장애인등록하고,국민연금관리공단은 장애3급으로 장애연금 받고 있어요~모르면 못 받고 알면 받고~  ㅎ영상 잘 봤습니다...




3 번째 동영상의 정보를 수집합니다.
댓글 20개
3 번째 동영상의 조회수는 24651 회 이고 수집할 댓글은 총 20개 입니다
3 번째 동영상의 제목은 질병후유장해 3% 꼭 필요한 보장일까? 입니다


3 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


3 번째 영상의 1 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=xg2Nz6ui854
2.댓글 작성자명: 팀보틀
3.댓글 작성일자: 4개월 전
궁금하신 부분과 연락처를 남겨주시면 상담을 진행해 드립니다..


3 번째 영상의 2 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=x

...

4.댓글 내용: 제보험은요 상해 재해휴유장애로 되있어요 제가 양쪽난소제거수술이 예약되있어요 보상받을수있을까요?


22 번째 영상의 49 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=MaLeumVM9_A
2.댓글 작성자명: 고건
3.댓글 작성일자: 4개월 전
4.댓글 내용: 죄송하지만 논리의오류가있어 답글남깁니다. 나이에따른후유장해율의안내에서 합산하실게아니라 각각으로보셔야 맞는내용이죠...




23 번째 동영상의 정보를 수집합니다.
댓글 0개
23 번째 동영상의 조회수는 1608 회 이고 수집할 댓글은 총 0개 입니다
23 번째 동영상의 제목은 [산재 장해등급 해설]  1화 뇌심혈관계질병과 장해등급의 관계 (뇌출혈, 뇌경색, 심근경색산재) 입니다


23 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~




24 번째 동영상의 정보를 수집합니다.
댓글 11개
24 번째 동영상의 조회수는 3269 회 이고 수집할 댓글은 총 11개 입니다
24 번째 동영상의 제목은 교통사고 후유장해평가후 보상 어렵지 않습니다(225회) 입니다


24 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요

3.댓글 작성일자: 1개월 전
4.댓글 내용: 오른 말씀입니다. 설명잘 들었습니다. 저도 아들 보험분석 하면서질병후유장해넣을러고하다가아직 않넣어슴니다.그런데 설계사중 책임지지도 못하면서 질명후유장해가장하다고 하는미친설계사가있내요 책임지고 후유장해받아줄까요.지식도없는 설계사한테속지마세요.질병후유장해  못받습니다. 생명회사 시알보험하고 비슷하다고 생각하시며되요.


28 번째 영상의 5 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=dZibGwCqYGE
2.댓글 작성자명: ms k
3.댓글 작성일자: 3개월 전
4.댓글 내용: 아이들 암,뇌,심 진단금외에 질병후유장애를 넣어야하나 고민중이었는데, 도움 많이 되었습니다. 감사합니다^^


28 번째 영상의 6 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=dZibGwCqYGE
2.댓글 작성자명: 동물사랑
3.댓글 작성일자: 3개월 전
4.댓글 내용: 상해후유장해는 어떤가요?건축건설업에 근무해서 위험성이 있을수 있어서..


28 번째 영상의 7 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=dZibGwCqYGE
2.댓글 작성자명: 박순경
3.댓글 작성일자: 1개월 전
4.댓글 내용: 조은정보감사합니다~^^


28 번째 영상의 8 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.y

4.댓글 내용: 항상 감사합니다


32 번째 영상의 4 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=ieRWeVZZ-5o
2.댓글 작성자명: Too TV-생활지식과 보험정보 공유채널
3.댓글 작성일자: 5개월 전
4.댓글 내용: 좋은 정보감사합니다.장애진단시점을 확인하는 방법을쉽게 확인할수록 있을 듯 합니다


32 번째 영상의 5 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=ieRWeVZZ-5o
2.댓글 작성자명: 일본이왜이렇게싫지
3.댓글 작성일자: 5개월 전
4.댓글 내용: �


32 번째 영상의 6 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=ieRWeVZZ-5o
2.댓글 작성자명: 파란하늘
3.댓글 작성일자: 5개월 전
4.댓글 내용: 감사합니다 ~~^^����


32 번째 영상의 7 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=ieRWeVZZ-5o
2.댓글 작성자명: 오현빈
3.댓글 작성일자: 5개월 전
4.댓글 내용: 장해진단서는 누구에게 받나요??


32 번째 영상의 8 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtu

2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


37 번째 영상의 1 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=fjE4obVj_pI
2.댓글 작성자명: 보상과배상TV
3.댓글 작성일자: 3개월 전
https://cafe.naver.com/pokemonguide.


37 번째 영상의 2 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=fjE4obVj_pI
2.댓글 작성자명: new빈
3.댓글 작성일자: 3개월 전
4.댓글 내용: 고도후유장애 보험금 받아본사람입니다 손해사정사분 잘알아보고 하셔야해요  저는 12년도 교통사고로 척추분쇄골절되서 5마디고정술하고 하반신마비도 있었어요 6개월후에 5마디고정한거와 운동범위제한인가로 53프로 후유장애보험금 나와서 재활치료받으며  생활하던중 15년도에 양쪽고관절괴사 진단받고 그원인이 사고나서 수술받기전부터 맞았던 고용량스테로이드로 인해서 괴사가 온거라 고관절 인공관절수술후 30프로 후유장애  합산후유장애가 된다는걸알고 인터넷검색으로 손해사정사 만나서 계약하고 일을맡겼어요 당시 스테로이드부작용으로 괴사뿐아니라 부신기능저하 호르몬이상과 골다공증  하반신통증심해서 마약성진통패치 사용중이었고 

3.댓글 작성일자: 3개월 전
4.댓글 내용: 저희 어머니도 작년6월5일 극심한 두통으로 병원을 방문후 병원에서 뇌출혈로 쓰러지시고 3개월동안 3번의 수술을 더 하시구. 쓰러지신 당일 잘못 쓰러지시면서 허리를 다쳤습니다. 뇌출혈로 수술하셔서 사망의 직전까지 가신분이라 허리치료를못해 허리다치신 그상태로 뼈가 굳어 전신마비의 가깝게 마비상태이십니다. 1년이 넘었지만 마비상태입니다. 병원에서는 더 이상 차도가 없으니 요양병원으로 옮기라고 하셨습니다. 대소변및 식사도 혼자 못하시고 24시간 365일 개인 간병이 필요한분입니다. 보험사 보험금청구를위해 담당선생님께 서류 발급을 받으면서 상해부터해서 후유장애100%를 받았습니다. 이 서류를 보험사에 제출을했더니. 뇌출혈 환자는 2년후에 자체 심사를 거쳐 그상태 그대로면 후유장애 보험금을 1억을 지불하고 그게 아닐시 지금이 안된다. 그때가서 2천받을래 아님 지금 2천받고 끝낼래라는 말을듣고 치료비가 급하기에 2천을 받고 서류에 사인을했습니다. 차후 약관을 보니 2년후에 심사를 거치는게 아니고 2년안으로 심사를 거쳐 보험지급이 된다 적혀있고. 고지 자체를 거짓으로 저희에게 안내를 했습니다. 녹취록도 가지고있고요. 이걸로 법적으로 싸움이 갈수있을까요.




38 번째 동영상의 정보를 수집합니다.
댓글 0개
38 번째 동영상의 조회수는 400 회 이고 수집할 댓글은 총 0개 입니다
38 번째 동영상의 제목은 척추압박골절 등통증과 후유장해 보상관계 입니다


38 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
